## This 

In [1]:
import pandas as pd
import numpy as np


In [2]:
def triage(resultList, sheetType): #analyse a row of the frame and return pass, review or fail, SheetType = morphology or topography (M or T)
    try:
        if resultList.get('fsn') == 'not found':  #this is the fully specified name column, = not found if snomed lookup failed
            return 'fail'
        if sheetType == 'M':  #for morph data 
            if (pd.isnull(resultList['Disease Type Id'])) or (pd.isnull(resultList['Disease Subtype Id'])):  #if there is no disease or subdisease code
                return 'review'
            elif max([resultList.get('Disease subtype match'), resultList.get('Disease subtype match words'), resultList.get('Synonym subtype match')]) < 50:   #these are the match scores: fail if highest <50, review if between 50-99, pass if = 100
                return 'fail'
            elif max([resultList.get('Disease subtype match'), resultList.get('Disease subtype match words'), resultList.get('Synonym subtype match')]) < 99:
                return 'review'
            elif max([resultList.get('Disease subtype match'), resultList.get('Disease subtype match words'), resultList.get('Synonym subtype match')]) == 100:
                return 'pass'
            elif resultList.get('semtag') != 'morphologic abnormality':
                return 'review'
        elif sheetType == 'T':  #for topo data
            if (pd.isnull(resultList['Disease Type Id'])):  #if there is no disease code
                return 'review'
            elif max([resultList.get('Disease match'), resultList.get('Disease match words'), resultList.get('Synonym disease match'), resultList.get('Parent term match')]) < 50:   #these are the match scores: fail if highest <50, review if between 50-99, pass if = 100
                return 'fail'
            elif max([resultList.get('Disease match'), resultList.get('Disease match words'), resultList.get('Synonym disease match'), resultList.get('Parent term match')]) < 99:
                return 'review'
            elif max([resultList.get('Disease match'), resultList.get('Disease match words'), resultList.get('Synonym disease match'), resultList.get('Parent term match')]) == 100:
                return 'pass'
            elif resultList.get('semtag') != 'body structure':
                return 'review'
    except:
        return 'fail'   #fail if values do not compute

In [10]:
filename = 'ICD RT CT codes 15Nov2016 v 2 annotated.xlsx'   #get the excel file annotated with scores

In [11]:
xl = pd.ExcelFile(filename)   #open the excel file

In [12]:
sheets = xl.sheet_names   #get the names of all sheets in the workbook
oldFrameList = []
for sheet in sheets:  #only analyse the 12 sheets with morph and topo data
    print (sheet)
    frame = xl.parse(sheet)   #make a dataframe from each sheet
    frame.name = sheet  #name the dataframe according to the sheet name
    oldFrameList.append(frame)   #add each frame to the list of frames

S-M-ICD
S-M-RT
S-M-CT
S-T-ICD
S-T-RT
S-T-CT
P-M-ICD
P-M-RT
P-M-CT
P-T-ICD
P-T-RT
P-T-CT


In [13]:
frameList = []
for frame in oldFrameList:
    oldName = frame.name
    if len(frame.columns.values) > 5:
        frame = frame[frame.term.notnull()].copy()
        frame.name = oldName
        frameList.append(frame)

In [14]:
for frame in frameList:
    if frame.name[2] == 'M':   #if the middle letter is M this is morphology data - use the morph triage
        frame['validation'] = frame.apply(triage, args = ('M',), axis=1)
    else:   #else use the topography triage
        frame['validation'] = frame.apply(triage, args = ('T',), axis=1)
         

In [15]:
writer = pd.ExcelWriter('ICD RT CT codes 15Nov2016 v 2 reviewed.xlsx', engine='xlsxwriter')

In [16]:
for sheet in frameList:
    sheet.to_excel(writer, index=False, sheet_name=sheet.name)
writer.save()